In [1]:
import json
import os
import re
import pandas as pd
import numpy as np
from sys import argv
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
import google.generativeai as genai 
import google.ai.generativelanguage as glm
from sentence_transformers import SentenceTransformer
import typing_extensions as typing
import enum

d:\Anaconda\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('./secrets.json') as f:
    secrets = json.loads(f.read())
GOOGLE_API_KEY = secrets['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)

safe = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

In [3]:
matching_keys = {
    '월요일이용비중': 'MON_UE_CNT_RAT',
    '화요일이용비중': 'TUE_UE_CNT_RAT',
    '수요일이용비중': 'WED_UE_CNT_RAT',
    '목요일이용비중': 'THU_UE_CNT_RAT',
    '금요일이용비중': 'FRI_UE_CNT_RAT',
    '토요일이용비중': 'SAT_UE_CNT_RAT',
    '일요일이용비중': 'SUN_UE_CNT_RAT',
    '5시~11시이용비중': 'HR_5_11_UE_CNT_RAT',
    '12시~13시이용비중': 'HR_12_13_UE_CNT_RAT',
    '14시~17시이용비중': 'HR_14_17_UE_CNT_RAT',
    '18시~22시이용비중': 'HR_18_22_UE_CNT_RAT',
    '23시~4시이용비중': 'HR_23_4_UE_CNT_RAT',
    '현지인이용건수비중': 'LOCAL_UE_CNT_RAT',
    '남성회원수비중': 'RC_M12_MAL_CUS_CNT_RAT',
    '여성회원수비중': 'RC_M12_FME_CUS_CNT_RAT',
    '20대이하회원수비중': 'RC_M12_AGE_UND_20_CUS_CNT_RAT',
    '30대회원수비중': 'RC_M12_AGE_30_CUS_CNT_RAT',
    '40대회원수비중': 'RC_M12_AGE_40_CUS_CNT_RAT',
    '50대회원수비중': 'RC_M12_AGE_50_CUS_CNT_RAT',
    '60대이상회원수비중': 'RC_M12_AGE_OVR_60_CUS_CNT_RAT'
}

In [4]:
emb_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
model = genai.GenerativeModel('gemini-1.5-flash-latest', safety_settings=safe)
csv_file_path = "final_coordinates_2.csv"
df = pd.read_csv(os.path.join('./data', csv_file_path),encoding='cp949')

d:\Anaconda\envs\llm\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [72]:
# 특정 요일 방문 횟수 비중을 나타내는 TypeDict
class VisitCountShare(typing.TypedDict):
    day: str          # 요일 (예: '월요일')
    visit_percentage: float  # 해당 요일의 방문 비중
    

class Category(enum.Enum):
    NONE = "None"
    Homestyle = "가정식"
    Chinese = "중식"
    SingleMenu = "단품요리 전문점"
    Coffee = "커피"
    IceCream = "아이스크림/빙수"
    Pizza = "피자"
    Western = "양식"
    Chicken = "치킨"
    Japanese = "일식"
    BeerAndPub = "맥주/요리주점"
    Sandwich = "샌드위치/토스트"
    LunchBox = "도시락"
    Bakery = "베이커리"
    KoreanSnacks = "분식"
    Skewers = "꼬치구이"
    Tea = "차"
    Steak = "스테이크"
    Cafeteria = "구내식당/푸드코트"
    AsianIndian = "동남아/인도음식"
    Hamburger = "햄버거"
    Ricecake = "떡/한과"
    FoodTruck = "포장마차"
    Juice = "주스"
    TraditionalPub = "민속주점"
    Buffet = "부페"
    WorldCuisine = "기타세계요리"
    Donut = "도너츠"
    TransportCafe = "기사식당"
    NightSnack = "야식"
    FamilyRestaurant = "패밀리 레스토랑"
    
# Enum은 이름이 아니라 내용을 보고 구분하는듯. 아마 직접 소비건수 상위 ... 이런식으로 하면 알아먹긴할듯
class Range(enum.Enum):
    NONE = "None"
    Top10Percent = "상위 10% 미만"
    Top10To25Percent = "상위 10%이상 25%미만"
    Top25To50Percent = "상위 25%이상 50%미만"
    Top50To75Percent = "상위 50%이상 75%미만"
    Top75To90Percent = "상위 75%이상 90%미만"
    Top90PercentOver = "상위 90% 초과"
    

class orderType(enum.Enum):
    NONE = "None"
    highest = "highest"
    lowest = "lowest"
    
class filterType(enum.Enum):
    NONE = "None"
    Mon = "월요일이용비중"
    Tue = "화요일이용비중"
    Wed = "수요일이용비중"
    Thu = "목요일이용비중"
    Fri = "금요일이용비중"
    Sat = "토요일이용비중"
    Sun = "일요일이용비중"
    HR_5_11 = "5시11시이용건수비중"
    HR_12_13 = "12시13시이용건수비중"
    HR_14_17 = "14시17시이용건수비중"
    HR_18_22 = "18시22시이용건수비중"
    HR_23_4 = "23시4시이용건수비중"
    Local = "현지인이용건수비중"
    Mal = "남성회원수비중"
    Fme = "여성회원수비중"
    Age_20_Und = "20대이하회원수비중"
    Age_30 = "30대회원수비중"
    Age_40 = "40대회원수비중"
    Age_50 = "50대회원수비중"
    Age_60_Ovr = "60대이상회원수비중"
    
# 필터와 정렬 정보를 위한 TypedDict
class FilterOrder(typing.TypedDict):
    filter_type: filterType  # 필터 종류 (예: 요일, 성별 등)
    order_type: orderType   # 정렬 타입 (예: 'highest', 'lowest')

class Query(typing.TypedDict):
    is_recommend: typing.Required[bool]
    address: str
    category: Category
    Usage_Count_Range: str
    Spending_Amount_Range: str
    Average_Spending_Amount_Range: str # Range쓰면 분류 잘못함.
    # Visit_count_specific: VisitCountShare
    # Local_Visitor_Proportion: float
    ranking_condition: FilterOrder
    
    
    
    
    

In [73]:
print(list(Query.__annotations__.keys())[:-1])

['is_recommend', 'address', 'category', 'Usage_Count_Range', 'Spending_Amount_Range', 'Average_Spending_Amount_Range']


In [89]:
# 고민할점. 대체로 많은 데이터가 뽑혔을때가 정확하다. 소수의 데이터의 정보는 무시할만 하지 않나(덮기)
chat = model.start_chat(history=[])
# question = "제주시 노형동에 있는 단품요리 전문점 중 이용건수가 상위 15%에 속하고 건당평균이용금액구간이 상위 20%에 속하면서 소비구간이 상위 80%이고 토요일 이용비중이 제일 낮은곳은?"
# question = "지금 공항앞인데 뭐 먹을거 없니?"
# question = "중문 숙성도처럼 숙성 고기 파는데 웨이팅은 적은 식당 있을까?"
# question = "서귀포시에 국밥집 괜찮은거 있나"
question = "제주시 한림읍에 있는 빵집 중 이용건수가 상위 15%에 속하고, 30대 이용 비중이 가장 높은 곳은?"
prompt = f"""질문에서 요구사항을 보고 JSON의 모든 항목(is_recommend, 주소, 업종, 이용건수구간, 이용금액구간, 건당평균이용금액구간)을 반드시 반환하라\n
        각 필드의 대한 설명이다. address:주소(예. 제주시 ㅁㅁ읍),    category:업종,   Usage_Count_Range:이용건수구간(예. 이용건수 상위 N%),  Spending_Amount_Range:이용금액구간(예. 이용금액구간 상위 N%),
        Average_Spending_Amount_Range:건당평균이용금액구간(예. 건당평균이용금액 상위 N%), is_recommend:(어떤식당을 추천해달라는 건가(True), 조건에 따른 검색인가(False))\n
        ranking_condition는 없을 수도 있으며, 오직 순위를 나타내는 조건(가장 큰것, 가장 작은것)에만 해당한다. 
        \n질문: {question}"""

response = chat.send_message(prompt, generation_config=genai.GenerationConfig(
        response_mime_type="application/json", response_schema=list[Query]
    ), )
# print(response)
output = response.parts[0].text
print(output)
json_datas = json.loads(response.parts[0].text)
print("객체 개수", len(json_datas))
if len(json_datas) > 1:
    print(json_datas[0])
    print(json_datas[1])

[{"Usage_Count_Range": "이용건수 상위 15%", "address": "제주시 한림읍", "category": "베이커리", "is_recommend": false, "ranking_condition": {"filter_type": "30대회원수비중", "order_type": "highest"}}]

객체 개수 1


In [8]:
# print(json_datas[2])

In [9]:
def merge_dicts(dicts):
    merged = {}

    for d in dicts:
        for key, value in d.items():
            if key == "ranking_condition" and key in merged:
                # ranking_condition의 filter_type이 None인 경우 넘어가고, 아니면 덮어씀
                if not value.get("filter_type"):
                    continue
                if value["filter_type"] != "None":
                    merged[key].update(value)
            else:
                # 다른 경우는 그냥 덮어씀
                merged[key] = value

    return merged

In [90]:
json_data = json.loads(response.parts[0].text)
result = merge_dicts(json_data)
print(result)

{'Usage_Count_Range': '이용건수 상위 15%', 'address': '제주시 한림읍', 'category': '베이커리', 'is_recommend': False, 'ranking_condition': {'filter_type': '30대회원수비중', 'order_type': 'highest'}}


In [11]:
# 상위 퍼센트 값(예: 14%, 79% 등)을 주어진 구간에 맞게 변환하는 함수
def map_to_group(raw_percentage):
    """
    사용자의 입력 퍼센티지 값을  구간으로 변환
    Args:
        percentage (float): 상위 퍼센티지 값 (예: 14.5, 79 등)

    Returns:
        str: 변환된 구간 값
    """
    if raw_percentage is None:
        return None
    
    pattern = r"(?:상위\s*)?([\d\.]+)%"
    match = re.search(pattern, raw_percentage)
    if match:
        percentage = float(match.group(1))
    else:
        return None
    
    output = None
    if 0 <= percentage < 10:
        output = '1_상위10%이하'
    elif 10 <= percentage < 25:
        output = '2_10~25%'
    elif 25 <= percentage < 50:
        output = '3_25~50%'
    elif 50 <= percentage < 75:
        output = '4_50~75%'
    elif 75 <= percentage < 90:
        output =  '5_75~90%'
    elif 90 <= percentage <= 100:
        output = '6_90% 초과(하위 10% 이하)'
    print(f"{raw_percentage}->{output}")
    return output
    

In [94]:
def filtering(dic):
    """일반 검색 대응

    Args:
        dic (dict): Json에서 가공된 Dictionary 데이터

    Returns:
        output (str): 출력할 텍스트
    """
    # 일반 쿼리 대응
    addr = dic.get("address", None)
    mct_type = dic.get("category", None)
    ranking_condition = dic.get('ranking_condition', {})
    filter_type = ranking_condition.get('filter_type', None)
    order_type = ranking_condition.get('order_type', None)
    
    # 형식 변환
    usage_Count_Range = map_to_group(dic.get("Usage_Count_Range", None))
    spending_Amount_Range = map_to_group(dic.get("Spending_Amount_Range", None))
    average_Spending_Amount_Range = map_to_group(dic.get("Average_Spending_Amount_Range", None))
    
    # 필터링
    conditions = []
    if addr is not None:
        conditions.append(df['ADDR'].str.contains(addr, na=False))
    if mct_type is not None:
        conditions.append(df['MCT_TYPE'].str.contains(mct_type, na=False))
    if usage_Count_Range is not None:
        conditions.append(df['UE_CNT_GRP'] == usage_Count_Range)
    if spending_Amount_Range is not None:
        conditions.append(df['UE_AMT_GRP'] == spending_Amount_Range)
    if average_Spending_Amount_Range is not None:
        conditions.append(df['UE_AMT_PER_TRSN_GRP'] == average_Spending_Amount_Range)
        
    if conditions:
        filtered_df = df.loc[pd.concat(conditions, axis=1).all(axis=1)]
    
    if filter_type != "None" and order_type != "None":
        is_ascending = (order_type == "lowest")
        filtered_df = filtered_df.sort_values(by=matching_keys.get(filter_type, None), ascending=is_ascending)
    
    output = None
    if len(filtered_df) == 0:
        output = "검색 결과가 없습니다."
    else:
        output = f"# 조건에 해당하는 식당을 찾았습니다.\n식당명: {filtered_df.iloc[0]['MCT_NM']}\n주소: {filtered_df.iloc[0]['ADDR']}"
    
    return output
    

In [87]:
def generate_prompt_from_json(law_data):
    # 받은 Json 데이터를 병합하고 질문 보완 프롬프트 생성(텍스트)
    dic = merge_dicts(json.loads(response.parts[0].text))

    # is_recommend 로 추천인지 검색인지 확인. 검색인것만 확실하면 2번 검색도 가능할것.
    if not dic.get("is_recommend", False):
        return filtering(dic)
    # 추천방식. 정확도가 필요없다. 아마 여기에 나이대, 성별 가중치를 적용할것(프롬프트로?)
    # 고민할점. 함수 호출 on/off가 상시로 가능한가.
    
    

In [93]:
print(generate_prompt_from_json(result))

이용건수 상위 15%->2_10~25%
# 조건에 해당하는 식당을 찾았습니다.
식당명: 집의기록1
주소: 제주 제주시 한림읍 귀덕리 951-1번지


In [95]:
output = response.parts[0].text
print(output)
print(response)

[{"Usage_Count_Range": "이용건수 상위 15%", "address": "제주시 한림읍", "category": "베이커리", "is_recommend": false, "ranking_condition": {"filter_type": "30대회원수비중", "order_type": "highest"}}]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "[{\"Usage_Count_Range\": \"\uc774\uc6a9\uac74\uc218 \uc0c1\uc704 15%\", \"address\": \"\uc81c\uc8fc\uc2dc \ud55c\ub9bc\uc74d\", \"category\": \"\ubca0\uc774\ucee4\ub9ac\", \"is_recommend\": false, \"ranking_condition\": {\"filter_type\": \"30\ub300\ud68c\uc6d0\uc218\ube44\uc911\", \"order_type\": \"highest\"}}]\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIB